Fetching data about houses from Sreality api

In [ ]:
import requests
import csv
import json
import re
import os

houses = []
total_pages = 454

for page in range(1, total_pages + 1):
    url = f"https://www.sreality.cz/api/cs/v2/estates?category_main_cb=2&category_type_cb=1&per_page=40&page={page}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        houses_page = data.get('_embedded', {}).get('estates', [])
        houses.append(houses_page)
        print(f"Načtena stránka {page}")
    else:
        print(f"Chyba při načítání stránky {page} (status code: {response.status_code})")

print(f"Celkem načteno {len(houses)} domů.")

Save all data to json file

In [ ]:
if(not os.path.exists("./data")):
    os.makedirs("../data")

with open('../data/houses.json', 'w') as f:
    json.dump(houses, f, indent=4)

Extract only needed data from json

In [ ]:
cleaned_houses = []

for i in houses:
    for house in i:
        price = house["price"]
        lat = house["gps"]["lat"]
        lon = house["gps"]["lon"]
        name = house["name"].replace("\u00a0", " ")
        numbers = re.findall(r'\d+(?:\s\d+)*', name)
        garage = 1 if "garage" in house["labelsAll"][0] else 0
        new = 1 if "new_building" in house["labelsAll"][0] else 0
        furnished = 1 if "furnished" in house["labelsAll"][0] else 0
        cellar = 1 if "cellar" in house["labelsAll"][0] else 0
        parkingLots = 1 if "parking_lots" in house["labelsAll"][0] else 0
        reconstructed = 1 if "after_reconstruction" in house["labelsAll"][0] else 0
        balcon = 1 if "balcony" in house["labelsAll"][0] else 0
        terrace = 1 if "terrace" in house["labelsAll"][0] else 0


        cleaned_houses.append([price,lat,lon,numbers[0],numbers[1],garage, new, furnished, cellar, parkingLots, reconstructed, balcon, terrace])
        print(len(cleaned_houses))

print("Domy jsou vyčištěny")

Save all extracted data to csv file

In [ ]:
with open("../data/houses.csv","w",newline='') as f:
        csvwriter = csv.writer(f)
        rows = ["price", "lat", "lon", "land_area","usable_area", "garage", "new", "furnished", "cellar", "parkingLots", "reconstructed", "balcon", "terrace"]
        csvwriter.writerow(rows)

for i in cleaned_houses:
     with open("../data/houses.csv","a",newline='') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerow(i)

After importing all data to database and using:
```SQL
SELECT garage, COUNT(*) FROM houses GROUP BY garage;
SELECT new, COUNT(*) FROM houses GROUP BY new;
SELECT furnished, COUNT(*) FROM houses GROUP BY furnished;
SELECT cellar, COUNT(*) FROM houses GROUP BY cellar;
SELECT parkingLots, COUNT(*) FROM houses GROUP BY parkingLots;
SELECT reconstructed, COUNT(*) FROM houses GROUP BY reconstructed;
SELECT balcon, COUNT(*) FROM houses GROUP BY balcon;
SELECT terrace, COUNT(*) FROM houses GROUP BY terrace;

```

This is the output:
- Garage
  - 0 - 11272
  - 1 - 6888
- New
  - 0 - 15732
  - 1 - 2428
- Furnished
  - 0 - 15842
  - 1 - 2318
- Cellar
  - 0 - 11513
  - 1 - 6647
- ParkingLots
  - 0 - 8697
  - 1 - 9463
- Reconstructed
  - 0 - 17086
  - 1 - 1074
- Balcon
  - 0 - 18153
  - 1 - 7
- Terrace
  - 0 - 18147
  - 1 - 13

This means tahta Balcon and Terrace are useless as they are almoust never there.